--- 
--- 
# Partie générative
--- 
--- 

Ce notebook contient notre modèle pour générer un texte original dans le cas où la question de l'utilisateur n'est pas orientée tâche.

---
## Importation des bibliothèques et modules
---

In [0]:
import tensorflow as tf
import numpy as np
import os
import time
import pandas as pd
import pickle

In [0]:
#pip install --upgrade tensorflow

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
with open("/content/gdrive/My Drive/OpenSubtitles.de-fr.fr" , encoding='utf-8') as f :
    subtitles = f.read()



---
## Approche par caractères
---



Avant d'entrainer le modèle, nous avons converti les chaines de caractères en numérique. Nous avons pour cela crée deux tables. L'une converti les caractères en nombres et la deuxième fait le processus inverse. Chaque caractère présent dans la base de texte OPUS a maintenant un nombre qui lui est attribué.

In [0]:
subtitles = subtitles[0:int(len(subtitles)/10)]

Nous avons appris sur seulement 20% de la base pour réduire le temps de calcul.

In [0]:
# Creating a mapping from unique characters to indices
vocab = sorted(set(subtitles))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in subtitles])

Nous enregistrons ces deux tables car nous en aurons besoin dans la partie principale qui rassemble les éléments du chatbot.

In [0]:
pickle.dump(char2idx, open('/content/gdrive/My Drive/lst_char.pkl','wb'))
pickle.dump(idx2char, open('/content/gdrive/My Drive/lst_idx.pkl','wb'))

---
# La prédiction
---

Maintenant en entrant une séquence de caractères (dans notre cas la question de l'utilisateur), nous devons déterminer quels caractères sont les plus probables pour dialoguer avec l'utilisateur. Nous allons pour cela utiliser un modèle RNN. Nous choisissons de faire un modèle qui retourne au maximum 100 caractères à l'utilisateur.

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = (len(subtitles)/10)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [15]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 223) # (batch_size, sequence_length, vocab_size)


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           57088     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 223)           228575    
Total params: 4,223,967
Trainable params: 4,223,967
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [18]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 223)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.4073415


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10

In [22]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 8642 steps
Epoch 1/10
8642/8642 [==============================] - 519s 60ms/step - loss: 1.2964
Epoch 2/10
8642/8642 [==============================] - 527s 61ms/step - loss: 1.1694
Epoch 3/10
8642/8642 [==============================] - 527s 61ms/step - loss: 1.1637
Epoch 4/10
8642/8642 [==============================] - 528s 61ms/step - loss: 1.1785
Epoch 5/10
8642/8642 [==============================] - 527s 61ms/step - loss: 1.4678
Epoch 6/10
8642/8642 [==============================] - 527s 61ms/step - loss: 1.9389
Epoch 7/10
8642/8642 [==============================] - 527s 61ms/step - loss: 1.9376
Epoch 8/10
8642/8642 [==============================] - 528s 61ms/step - loss: 1.6646
Epoch 9/10
8642/8642 [==============================] - 527s 61ms/step - loss: 1.3502
Epoch 10/10
8642/8642 [==============================] - 525s 61ms/step - loss: 1.2752


In [23]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            57088     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 223)            228575    
Total params: 4,223,967
Trainable params: 4,223,967
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.save('/content/gdrive/My Drive/generative_model.h5')